In [2]:
DEARGADAOL_LANDING = "https://www.leighleat.com/sc%C3%A9alta/45"

In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
def get_page_list(url: str):
    PAGE_REQ = requests.get(url)
    if PAGE_REQ.status_code != 200:
        return None

    soup = BeautifulSoup(PAGE_REQ.content, "html.parser")

    pages = set()
    for row in soup.find_all("div", {'class': 'row'}):
        for anchor in row.find_all("a"):
            if anchor["href"].startswith("/pages/"):
                pages.add(f'https://www.leighleat.com{anchor["href"]}')
    return list(pages)

In [5]:
def get_page(url: str, getnext: bool = False):
    PAGE_REQ = requests.get(url)
    if PAGE_REQ.status_code != 200:
        return None
    output = {}
    output["url"] = url

    soup = BeautifulSoup(PAGE_REQ.content, "html.parser")

    audio_url = ""
    audio = soup.find("audio", {"id": "audio"})
    if audio:
        audio_source = audio.find("source")
        if audio_source:
            if audio_source["src"].startswith("http"):
                audio_url = audio_source["src"]
            elif audio_source["src"].startswith("/"):
                audio_url = f'https://www.leighleat.com{audio_source["src"]}'

    text_pieces = []
    content = soup.find("div", {"class": "story-page-content"})
    selector = "h1, p"
    next_selector = "a.next-btn"
    if not content:
        content = soup.find("div", {"class": "inner-container"})
        selector = "h1.page-title, div.text > p"
    if not content:
        content = soup.find("div", {"class": "mobile-page-text"})
        selector = "h2, p"
    if not content:
        content = soup.find("div", {"class": "g-hidden-lg-down"})
        selector = "div.page-text > h1, div.page-text > p"
    if content:
        for text in content.select(selector):
            if text.text.strip() != "":
                text_pieces.append(text.text.strip())
    
    if getnext:
        next_button = soup.select_one(next_selector)
        output["next"] = f'https://www.leighleat.com{next_button["href"]}'

    output["audio"] = audio_url
    output["text"] = text_pieces

    return output
    

In [6]:
COUNT_DIGITS = {
    1: "a haon",
    2: "a dó",
    3: "a trí",
    4: "a ceathair",
    5: "a cúig",
    6: "a sé",
    7: "a seacht",
    8: "a hocht",
    9: "a naoi"
}

In [7]:
DIALECT_MU = {
    "anseo": "anso",
    "ansin": "ansan",
    "dtús": "dtúis",
    "tosú": "tosnú",
    "tosóidh": "tosnóidh",
    "atá": "athá",
    "atá tú": "athánn tú",
    "léi": "léithe",
    "arís": "aríst",
    "cloisteáil": "cloisint",
    "faoi": "fé",
    "an-suim": "an-shuim",
    "scríofa": "scríte",
}

In [8]:
PAGE_SPECIFIC = {
    "181": {
        "Mar 'Tháinig": "Mar a Tháinig",
        "Cabidil": "Caibidil",
        "mar Chuir": "mar a Chuir"
    },
    "190": {
        "nuair d'fhéach": "nuair a d'fhéach"
    },
    "191": {
        "le mo": "lem"
    }
}

In [9]:
def skip_if_colon(text):
    if ":" in text:
        return ""
    else:
      return text

def skip_art(text):
    if text.startswith("(Ealaín:"):
        return ""
    else:
        return text

In [10]:
get_page("https://www.leighleat.com/pages/548", True)

{'url': 'https://www.leighleat.com/pages/548',
 'next': 'https://www.leighleat.com/pages/549',
 'audio': 'https://www.leighleat.com/rails/active_storage/blobs/redirect/eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaHBBdklWIiwiZXhwIjpudWxsLCJwdXIiOiJibG9iX2lkIn19--aff8cb94bc79129d01554b64d74c85047f42c44f/lth%202.mp3',
 'text': ['Mianta… nó Mallachtaí?',
  'Fadó, fadó, rinne Janus dhá leabhar draíochta. Dia ab ea Janus a raibh maith agus olc ann. Thug Leabhar na Mianta gach mian duit – gach rud a theastaigh uait. Ach d’imir Janus cleas ar na daoine. Rinne sé an dara leabhar. Bhí sé an-chosúil leis an gcéad leabhar ach gur chuir sé do chuid mianta trí chéile. Ba é seo Leabhar na Mallachtaí. B’fhéidir gur léigh tú faoi cheana. Agus b’fhéidir go gcuirfeadh sé iontas ort a chloisteáil go raibh Leabhar na Mianta níos measa agus níos contúirtí ná Leabhar na Mallachtaí.']}

In [13]:
get_page("https://www.leighleat.com/sc%C3%A9alta/15", True)

{'url': 'https://www.leighleat.com/sc%C3%A9alta/15',
 'next': 'https://www.leighleat.com/pages/88',
 'audio': 'https://www.leighleat.com/rails/active_storage/blobs/redirect/eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaHBBazRCIiwiZXhwIjpudWxsLCJwdXIiOiJibG9iX2lkIn19--32cb80d4bafc92c56929a3b14b8c03f36494793e/An%20Leon%20Baineann%20p.1.mp3',
 'text': ['AN LEON BAINEANN',
  'Bhí an-iomaíocht idir ainmhithe na coille go léir féachaint cé acu a bhí ina máthair ar an ál ba mhó. \nTugadh na máithreacha go léir isteach ina gceann agus ina gceann agus cuireadh an cheist orthu faoi líon a n-ál.']}